# This kernel use simple transfer learning to detect ship exist or not
* For biginner, can get about 85% accuracy train on 10 epoch (~20min one epoch no GPU, ~100s one epoch have GPU)

Use ResNet50 to do transfer learning  
Load 5000 picture to be training data  
Split 4000 training set , 1000 validate set (0.2%)  
Image size 256 x 256, RGB data  
Using ImageGenerator to do data augumatation  

https://www.kaggle.com/super13579/simple-transfer-learning-detect-ship-exist-keras

# Load segmentation file


In [2]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
#from skimage.util.montage import montage2d as montage
path = "/mnt/3CE35B99003D727B/input/kaggle/Airbus Ship Detection Challenge/"


import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


train = pd.read_csv(os.path.join(path + 'train_ship_segmentations_v2.csv'))

print(train.shape[0],'masks found')

train.head()

231723 masks found


,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


# Tranfer EncodedPixels to target¶
have ship ==> 1  
No ship ==> 0

In [3]:
train['exist_ship'] = train['EncodedPixels'].fillna(0)
train.loc[train['exist_ship']!=0,'exist_ship']=1
del train['EncodedPixels']

In [4]:
train

,ImageId,exist_ship
0,00003e153.jpg,0
1,0001124c7.jpg,0
2,000155de5.jpg,1
3,000194a2d.jpg,1
4,000194a2d.jpg,1
...,...,...
231718,fffedbb6b.jpg,0
231719,ffff2aa57.jpg,0
231720,ffff6e525.jpg,0
231721,ffffc50b4.jpg,0


# We found there are some duplicate image in training data
groupby duplicate image

In [5]:
print(len(train['ImageId']))
print(train['ImageId'].value_counts().shape[0])
train_gp = train.groupby('ImageId').sum().reset_index()
train_gp.loc[train_gp['exist_ship']>0,'exist_ship']=1

231723
192556


# Balance have chip and no chip data
Remove 100000 data of no chip

In [6]:
print(train_gp['exist_ship'].value_counts())
train_gp= train_gp.sort_values(by='exist_ship')
train_gp = train_gp.drop(train_gp.index[0:100000])

0    150000
1     42556
Name: exist_ship, dtype: int64


In [7]:
train_gp.shape

(92556, 2)

In [8]:
train_gp

,ImageId,exist_ship
41054,36ca35038.jpg,0
41029,36c3a586e.jpg,0
41027,36c37bb3f.jpg,0
41005,36bc3ace0.jpg,0
41007,36bcf7e37.jpg,0
...,...,...
152664,cb07f89a0.jpg,1
65439,5724346f7.jpg,1
31034,29377f734.jpg,1
152654,cb02925f6.jpg,1


# Set Training set count
prevent large data cause much time

In [9]:
print(train_gp['exist_ship'].value_counts())
train_sample = train_gp.sample(5000)
print(train_sample['exist_ship'].value_counts())
print (train_sample.shape)

0    50000
1    42556
Name: exist_ship, dtype: int64
0    2714
1    2286
Name: exist_ship, dtype: int64
(5000, 2)


# Load training data function¶
load training data to numpy array for training

In [11]:
Train_path = path + 'train_v2/'
Test_path = path + 'test_v2/'

In [12]:
Train_path

'/mnt/3CE35B99003D727B/input/kaggle/Airbus Ship Detection Challenge/train_v2/'

In [13]:
%%time
training_img_data = []
target_data = []
from PIL import Image
data = np.empty((len(train_sample['ImageId']),256, 256,3), dtype=np.uint8)
data_target = np.empty((len(train_sample['ImageId'])), dtype=np.uint8)
image_name_list = os.listdir(Train_path)
index = 0
for image_name in image_name_list:
    if image_name in list(train_sample['ImageId']):
        imageA = Image.open(Train_path+image_name).resize((256,256)).convert('RGB')
        data[index]=imageA
        data_target[index]=train_sample[train_gp['ImageId'].str.contains(image_name)]['exist_ship'].iloc[0]
        index+=1
        
print(data.shape)
print(data_target.shape)

/home/jake/venv/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


(5000, 256, 256, 3)
(5000,)
CPU times: user 4min 13s, sys: 1.1 s, total: 4min 14s
Wall time: 6min 2s


# Doing One hot on target¶
Set target to one hot target for classification problem

In [16]:
data.shape

(5000, 256, 256, 3)

In [22]:
data_target.shape

(5000,)

# Doing One hot on target
Set target to one hot target for classification problem

In [20]:
from sklearn.preprocessing import OneHotEncoder
targets =data_target.reshape(len(data_target),-1)
enc = OneHotEncoder()
enc.fit(targets)
targets = enc.transform(targets).toarray()
print(targets.shape)

(5000, 2)


In [25]:
targets.shape

(5000, 2)

In [26]:
targets

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

# Split Training data to training data and validate data to detect overfit


In [27]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data,targets, test_size = 0.2)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((4000, 256, 256, 3), (1000, 256, 256, 3), (4000, 2), (1000, 2))

# Data augumatation
Using ImageDataGenerator

In [29]:
from keras.preprocessing.image import ImageDataGenerator
img_gen = ImageDataGenerator(
    rescale=1./255,
    zca_whitening = False,
    rotation_range = 90,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    brightness_range = [0.5, 1.5],
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True
    
)

# Load ResNet50 model with Keras
on Kaggle kernel, please turn on the internet setting to Internet connect on right windowLoad ResNet50 model with Keras  
on Kaggle kernel, please turn on the internet setting to Internet connect on right window  

In [32]:
#from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
#from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
from keras.applications.resnet50 import ResNet50 as ResModel
#from keras.applications.vgg16 import VGG16 as VGG16Model
img_width, img_height = 256, 256
model = ResModel(weights = 'imagenet', include_top=False, input_shape = (img_width, img_height, 3))

/home/jake/venv/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a268eb855778b3df3c7506639542a6af so we will re-download the data.
94658560/94653016 [==============================] - 182s 2us/step


# Add fully connect layer¶
Freeze convolution layer and add fully connect layer   
On this case, we only need predict 2 category (1. have ship, 2. no ship)  
For transfer learning, we only need to train parametric on fully connect layer  

In [33]:
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model 
for layer in model.layers:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

/home/jake/venv/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


# Set Hyperparameter and Start training
SGD optimizer  
Using categorical_crossentropy to be loss function  
lrate set to 0.001 (Maybe we have better value, In here, I have no experence on this  

In [34]:
from keras import optimizers
epochs = 10
lrate = 0.001
decay = lrate/epochs
#adam = optimizers.Adam(lr=lrate,beta_1=0.9, beta_2=0.999, decay=decay)
sgd = optimizers.SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model_final.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model_final.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [35]:
model_final.fit_generator(img_gen.flow(x_train, y_train, batch_size = 16),steps_per_epoch = len(x_train)/16, validation_data = (x_val,y_val), epochs = epochs )
model_final.save(path + '/models/ResNet_transfer_ship.h5')

Epoch 1/10
250/250 [==============================] - 48s 191ms/step - loss: 1.1158 - accuracy: 0.7097 - val_loss: 0.4400 - val_accuracy: 0.8250
Epoch 2/10
250/250 [==============================] - 43s 174ms/step - loss: 0.4739 - accuracy: 0.7905 - val_loss: 0.4375 - val_accuracy: 0.8120
Epoch 3/10
250/250 [==============================] - 43s 173ms/step - loss: 0.4591 - accuracy: 0.7912 - val_loss: 0.4101 - val_accuracy: 0.8350
Epoch 4/10
250/250 [==============================] - 43s 170ms/step - loss: 0.4453 - accuracy: 0.7947 - val_loss: 0.4090 - val_accuracy: 0.8210
Epoch 5/10
250/250 [==============================] - 43s 171ms/step - loss: 0.4192 - accuracy: 0.8092 - val_loss: 0.3915 - val_accuracy: 0.8400
Epoch 6/10
250/250 [==============================] - 43s 170ms/step - loss: 0.4014 - accuracy: 0.8260 - val_loss: 0.4042 - val_accuracy: 0.8350
Epoch 7/10
250/250 [==============================] - 43s 172ms/step - loss: 0.4026 - accuracy: 0.8140 - val_loss: 0.4058 - val_ac

In [37]:
import gc
gc.collect()

35

# Predict accuracy by random read training data
Get random 2000 data from training set

In [38]:
train_predict_sample = train_gp.sample(2000)
print(train_predict_sample['exist_ship'].value_counts())

0    1062
1     938
Name: exist_ship, dtype: int64


In [39]:
%%time
from PIL import Image
data_predict = np.empty((len(train_predict_sample['ImageId']),256, 256,3), dtype=np.uint8)
data_target_predict = np.empty((len(train_predict_sample['ImageId'])), dtype=np.uint8)
image_name_list = os.listdir(Train_path)
index = 0
for image_name in image_name_list:
    if image_name in list(train_predict_sample['ImageId']):
        imageA = Image.open(Train_path+image_name).resize((256,256)).convert('RGB')
        data_predict[index]=imageA
        data_target_predict[index]=train_predict_sample[train_gp['ImageId'].str.contains(image_name)]['exist_ship'].iloc[0]
        index+=1
        
print(data_predict.shape)
print(data_target_predict.shape)

/home/jake/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


(2000, 256, 256, 3)
(2000,)
CPU times: user 1min 43s, sys: 528 ms, total: 1min 44s
Wall time: 2min 24s


# Do one hot for predict target



In [40]:
from sklearn.preprocessing import OneHotEncoder
targets_predict =data_target_predict.reshape(len(data_target_predict),-1)
enc = OneHotEncoder()
enc.fit(targets_predict)
targets_predict = enc.transform(targets_predict).toarray()
print(targets_predict.shape)

(2000, 2)


# Evaluate predict


In [41]:
predict_ship = model_final.evaluate(data_predict,targets_predict)

2000/2000 [==============================] - 9s 4ms/step


In [42]:
print ('Accuracy of random data = '+ str(round(predict_ship[1]*100)) + "%")

Accuracy of random data = 88%


In [43]:
image_test_name_list = os.listdir(Test_path)
data_test = np.empty((len(image_test_name_list),256, 256,3), dtype=np.uint8)
test_name = []
index = 0
for image_name in image_test_name_list:
    imageA = Image.open(Test_path+image_name).resize((256,256)).convert('RGB')
    test_name.append(image_name)
    data_test[index]=imageA
    index+=1
print (data_test.shape)

(15606, 256, 256, 3)


In [44]:
result = model_final.predict(data_test)

In [45]:
result_list={
    "ImageId": test_name,
    "Have_ship":np.argmax(result,axis=1)
}
result_pd = pd.DataFrame(result_list)
result_pd.to_csv('Have_ship_or_not.csv',index = False)